In [ ]:
"""import sys
from pathlib import Path
cwd = Path.cwd()
if (cwd / 'src').exists():
    ROOT = cwd
else:
    ROOT = cwd.parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))
if str(ROOT / 'src') not in sys.path:
    sys.path.insert(0, str(ROOT / 'src'))

import importlib
importlib.invalidate_caches()

# Install pysindy if needed: `pip install pysindy`
import numpy as np
print("Add code to prepare X matrix and run SINDy")"""

In [10]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np

# Go up one directory from notebooks/
ROOT = Path.cwd().parent

# Add project root to PYTHONPATH
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

print("PYTHONPATH updated with:", ROOT)


from src.ml.sindy_skeleton import sindy_fit


PYTHONPATH updated with: d:\OneDrive\Documents\My Docs\CMU\Fall '25\Model & Sims\Project\ODE-to-Circadian-Clocks


In [11]:
PROC = ROOT/ "notebooks"  / "data" / "processed"

train_df = pd.read_csv(PROC / "pca_train.csv")
val_df   = pd.read_csv(PROC /"pca_val.csv")
test_df  = pd.read_csv(PROC / "pca_test.csv")

pc_cols = [c for c in train_df.columns if c.startswith("PC")]


In [12]:
# Concatenate time series from all training subjects
X_train = train_df[pc_cols].values
t_train = train_df["t_idx"].values.astype(float)

# If needed: enforce uniform Δt
t = np.linspace(t_train.min(), t_train.max(), len(t_train))


In [13]:
model = sindy_fit(
    X=X_train,
    t=t,
    threshold=0.01,
    smooth=True
)

model.print()


c:\Users\Sanchitha\anaconda3\envs\scanpy-env\lib\site-packages\pysindy\utils\axes.py:389: RuntimeWarning: invalid value encountered in power
  results = super().__array_ufunc__(ufunc, method, *args, **kwargs)


MemoryError: Unable to allocate 7.12 GiB for an array with shape (177, 5397487) and data type float64

In [11]:
x0 = X_train[0]

t_sim = np.linspace(t.min(), t.max(), 200)
X_sim = model.simulate(x0, t_sim)

plt.figure(figsize=(10,6))
plt.plot(t, X_train[:,0], label="True PC1")
plt.plot(t_sim, X_sim[:,0], '--', label="SINDy PC1 Pred")
plt.legend()
plt.xlabel("Circadian Time Index")
plt.ylabel("PC1")
plt.title("SINDy Simulation vs True PCA Trajectory")
plt.show()


NameError: name 'model' is not defined

In [ ]:
# Interpolate simulation into val time-grid
from scipy.interpolate import interp1d

sim_func = interp1d(t_sim, X_sim[:,0], fill_value="extrapolate")

val_pred = sim_func(val_df["t_idx"].values)
val_true = val_df["PC1"].values

rmse = np.sqrt(np.mean((val_pred - val_true)**2))
rmse
